In [19]:
import pandas as pd
from awesometel.utils import load
from datetime import datetime

In [33]:
dfs = load()
df_products = dfs["product_holdings"]
df_products['valid_to'] = pd.to_datetime(df_products['valid_to'], errors="coerce", format='%Y-%m-%d').dt.date
df_products['valid_from'] = pd.to_datetime(df_products['valid_from'], errors="coerce", format='%Y-%m-%d').dt.date

In [39]:
# Detect active subscriptions
current_date = datetime.today().date()
# Consider both dates in the future and NaT as active subscriptions
df_products['active'] = (df_products['valid_to'] >= current_date) | df_products['valid_to'].isna()

# Group by account_id to detect churn
account_status = df_products.groupby('account_id').agg({
    "active": "sum",
    "valid_to": lambda x: x.dropna().max()
}).reset_index()
account_status['churned'] = account_status['active'] == 0

In [43]:
account_status

,account_id,active,valid_to,churned
0,1000,1,2022-06-08,False
1,1001,5,2021-11-09,False
2,1002,1,2022-08-22,False
3,1003,1,2022-03-07,False
4,1004,0,2022-07-16,True
...,...,...,...,...
8933,9995,1,2023-03-06,False
8934,9996,2,2023-03-09,False
8935,9997,3,2022-07-15,False
8936,9998,3,NaN,False


In [47]:
account_status.groupby("churned").size()

churned
False    6985
True     1953
dtype: int64

In [48]:
df_products.shape

(24517, 8)

# Check 1

In [61]:
df_accounts_info = dfs["account_info"]
df_interactions = dfs["interactions"]

In [63]:
df_merged = df_accounts_info.merge(account_status, how="left", on="account_id")
df_merged = df_merged.merge(df_interactions, how="left", on="account_id")

In [68]:
df_merged.loc[df_merged["churned"] == True]

8        True
9        True
10       True
18       True
19       True
         ... 
16363    True
16365    True
16366    True
16367    True
16369    True
Name: churned, Length: 3482, dtype: object

,account_id,date,time_in_queue,handling_time_s,call_reason,customer_satisfaction_after_call
0,1000,2023-07-02,26,36,Technical,NaN
1,1000,2023-11-11,8,52,Billing,NaN
2,1001,2021-08-05,30,114,Billing,NaN
3,1001,2023-05-27,14,41,Technical,NaN
4,1001,2021-10-31,28,64,Technical,5.0
...,...,...,...,...,...,...
14685,9995,2023-04-16,48,105,Product,NaN
14686,9996,2021-09-21,11,80,Billing,1.0
14687,9996,2023-12-04,72,129,Product,4.0
14688,9997,2022-12-25,28,72,Billing,4.0
